In [62]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import math
from scipy.ndimage import filters
from scipy.stats import multivariate_normal

In [91]:
octaves = 3
scales = 6
sigma = 1.25
contrastThreshold = 7.68
rCurvature = 10

array_of_img = [] # Store all the image data

In [122]:
# Read directory function
def read_directory(directory_name):
    filenumber = len([name for name in os.listdir(directory_name) if os.path.isfile(os.path.join(directory_name, name))])
    for i in range(1,filenumber+1):
        img = cv2.imread(directory_name + "/" + str(i)+ ".jpg")
        #size = img.shape
        #img = cv2.resize(img, (int(size[1]//10),int(size[0]//10)), interpolation=cv2.INTER_AREA)
        array_of_img.append(img)

In [93]:
def Rround(x):
    if x-np.floor(x) >=0.5:
        x = np.floor(x)+1
    else:
        x = np.floor(x)
    return x

In [94]:
def find_matches(fA, fB):
    rMatches = []
    print((len(fA), len(fB)))
    f_distance = np.zeros((len(fA), len(fB)), dtype = np.float)
    for i in range(0, len(fA)):
        itoj = [(0,0,0),(0,0,0)]
        for j in range(0, len(fB)):
            f_distance[i][j] = np.sqrt(np.sum((fA[i][:] - fB[j][:])**2))
            if itoj[0][2] == 0:
                itoj = [(i,j,f_distance[i][j]),(0,0,0)]
            elif itoj[1][2] == 0:
                if f_distance[i][j] < itoj[0][2]:
                    itoj[1] = itoj[0]
                    itoj[0] = (i,j,f_distance[i][j])
                else:
                    itoj[1] = (i,j,f_distance[i][j])
            else:
                if f_distance[i][j] <  itoj[0][2]:
                    itoj[1] = itoj[0]
                    itoj[0] = (i,j,f_distance[i][j])
                elif f_distance[i][j] <  itoj[1][2]:
                    itoj[1] = (i,j,f_distance[i][j])
        rMatches.append(itoj)
    matches = []
    for m in rMatches:
        #print(m)
        if len(m) == 2 and m[0][2] < m[1][2]*0.8:
            matches.append((m[0][0], m[0][1]))
    #print(rMatches)
    print(len(matches))
    return(matches)

In [95]:
def findHomo2(ptsA, ptsB):#A is right B is left
    size = len(ptsA)
    recor = [0,[1,0,0,1,0,0]]
    for i in range(0, size-1):
        for j in range(i+1, size):
            count = 0
            count2 = 0
            a = 0
            b = 0
            ca1 = 0
            ca2 = 0
            cb1 = 0
            cb2 = 0
            if (ptsA[j][1] - ptsA[i][1]) != 0 and (ptsA[j][0] - ptsA[i][0]) != 0:
                b = (float)((ptsB[j][0] - ptsB[i][0]) - (ptsA[j][0] - ptsA[i][0])) / (ptsA[j][1] - ptsA[i][1])
                a = (float)((ptsB[j][1] - ptsB[i][1]) - (ptsA[j][1] - ptsA[i][1])) / (ptsA[j][0] - ptsA[i][0])
                ca1 = ptsB[i][0] - ptsA[i][0] - a*ptsA[i][1]
                cb1 = ptsB[i][0] - ptsA[i][0] - b*ptsA[i][1]
                ca2 = ptsB[i][1] - ptsA[i][1] - a*ptsA[i][0]
                cb2 = ptsB[i][1] - ptsA[i][1] - b*ptsA[i][0]
            #if c2 < -100:
            #    continue
            else:
                continue
            m = np.array([[1, a],[a, 1],[ca1, ca2]])
            m2 = np.array([[1, b],[b, 1],[cb1, cb2]])
            for h in range(0, size):
                p = np.dot([ptsA[h][0], ptsA[h][1],1], m)
                p2 = np.dot([ptsA[h][0], ptsA[h][1],1], m2)
                if (p[0] - ptsB[h][0])**2+(p[1] - ptsB[h][1])**2 < 9 :
                    count += 1
                if (p2[0] - ptsB[h][0])**2+(p2[1] - ptsB[h][1])**2 < 9 :
                    count2 += 1
            if recor[0] < count:
                recor = [count,m.ravel()]
            if recor[0] < count2:
                recor = [count2,m2.ravel()]
                #print("change")
    H = np.array([[recor[1][0], recor[1][2], recor[1][4]],[recor[1][1], recor[1][3], recor[1][5]],[0,0,1]])

    return H

In [96]:
def findHomo(kA, kB, fA, fB, bound, re):
    matches = find_matches(fA, fB)
    if len(matches) > 3:
        ptsA = []
        ptsB = []
        for x,y in matches:
            if abs(kA[x][0] - kB[y][0]) > 50:
                continue
            if kB[y][1] < bound/3 or kA[x][1] > 2*bound/3:
                continue
            if kB[y][1] < bound/2 and kA[x][1] > bound/2:
                continue
            ptsA.append((kA[x][0]*re,kA[x][1]*re))
            ptsB.append((kB[y][0]*re,kB[y][1]*re))
        Homo = findHomo2(ptsA, ptsB)
        #print(Homo)
        return Homo
    else:#to be changed
        ptsA = np.float32([kA[x] for (x, _) in matches])
        ptsB = np.float32([kB[x] for (_, x) in matches])
        b = (float)((ptsB[1][0] - ptsB[0][0]) - (ptsA[1][0] - ptsA[0][0])) / (ptsA[1][1] - ptsA[0][1])
        a = (float)((ptsB[1][1] - ptsB[0][1]) - (ptsA[1][1] - ptsA[0][1])) / (ptsA[1][0] - ptsA[0][0])
        c1 = ptsB[0][0] - ptsA[0][0] - b*ptsA[0][1]
        c2 = ptsB[0][1] - ptsA[0][1] - a*ptsA[0][0]
        return np.array([[1,b,c1],[a,1,c2],[0,0,1]])


In [118]:
def warpTransport2(imgA, imgB, Homo, bound): #y|_x
    parti = 30
    size = imgA.shape#(y,x)
    nsize = np.dot([imgA.shape[0], imgA.shape[1], 1],Homo)#[y,x,1] = [y,x,1]*Homo
    #print(nsize)
    if nsize[0] < bound:#y
        nsize[0] = bound
    else:
        nsize[0] = Rround(nsize[0])+1
    nsize[1] = imgA.shape[1] + imgB.shape[1]#x
    print(nsize)
    nimg = np.zeros((int(nsize[0]), int(nsize[1]), 3))
    dusize = np.dot([0, 11, 1],Homo)#[y,x,1] = [y,x,1]*Homo
    drift = dusize[0]/(2*parti)
    #print(drift)
    for j in range(0, parti+2):#x
        for i in range(0, imgA.shape[0]):#y
            pa = np.dot([i, j, 1],Homo)#[y,x,1] = [y,x,1]*Homo
            p = (int(Rround(pa[1])), int(Rround(pa[0])))#x,y
            if p[1] >= imgB.shape[0] or p[0] >= imgB.shape[1] or p[1] < 0 or p[0] < 0:# or py >= int(nsize[0])
                continue
            if drift != 0:
                py = p[1]
                if j == 0:
                    if i == 0 and p[1] > 0:
                        nimg[:p[1]+1,:p[0]+1,:] = imgB[:p[1]+1,:p[0]+1,:]
                        
                    else:
                        if p[0] < imgB.shape[1] and p[1] < imgB.shape[0]:
                            nimg[p[1],:p[0]+1,:] = imgB[p[1],:p[0]+1,:]
                        elif p[1] < imgB.shape[0]:
                            nimg[p[1],:imgB.shape[1],:] = imgB[p[1],:imgB.shape[1],:]
                        
                elif j <= parti:
                    py = p[1] - int(drift*j)
                    if py > 0:
                        nimg[py,p[0],:] = imgA[i,j,:]*(float)(j)/parti + imgB[p[1],p[0],:]*(float)(parti-j)/parti
                    #else:
                    #    print("no put on "+str(p))
                else:
                    py = p[1] - int(drift*parti)
                    if py > 0:
                        nimg[py,p[0]:imgA.shape[1]-j+p[0],:] = imgA[i,j:,:]
                    else:
                        print("no put on "+str(p))
                if p[0] > 2 and max(nimg[py,p[0]-1,:]) == 0:
                    if max(nimg[py,p[0]-2,:])!=0:
                        nimg[py,p[0]-1,:] = nimg[py,p[0],:]*0.5 + nimg[py,p[0]-2,:]*0.5
                    elif p[0] > 3 and max(nimg[py,p[0]-3,:])!=0:
                        nimg[py,p[0]-2,:] = nimg[py,p[0]-3,:]
                        nimg[py,p[0]-1,:] = nimg[py,p[0],:]*0.5 + nimg[py,p[0]-2,:]*0.5
                    #else:
                    #    print("no interp y,x:"+str(py-1) +","+str(p[0]))
            else:
                if j == 0:
                    if i == 0 and p[1] > 0:
                        nimg[:p[1]+1,:p[0]+1,:] = imgB[:p[1]+1,:p[0]+1,:]
                    else:
                        if p[0] < imgB.shape[1] and p[1] < imgB.shape[0]:
                            nimg[p[1],:p[0]+1,:] = imgB[p[1],:p[0]+1,:]
                        elif p[1] < imgB.shape[0]:
                            nimg[p[1],:imgB.shape[1],:] = imgB[p[1],:imgB.shape[1],:]
                elif j <= parti:
                    nimg[p[1],p[0],:] = imgA[i,j,:]*(float)(j)/parti + imgB[p[1],p[0],:]*(float)(parti-j)/parti
                else:
                    nimg[p[1],p[0]:imgA.shape[1]-j+p[0],:] = imgA[i,j:,:]
                if p[0] > 2 and max(nimg[p[1],p[0]-1,:]) == 0:
                    if max(nimg[p[1],p[0]-2,:])!=0:
                        nimg[p[1],p[0]-1,:] = nimg[p[1],p[0],:]*0.5 + nimg[p[1],p[0]-2,:]*0.5
                    elif p[0] > 3 and max(nimg[p[1],p[0]-3,:])!=0:
                        nimg[p[1],p[0]-2,:] = nimg[p[1],p[0]-3,:]
                        nimg[p[1],p[0]-1,:] = nimg[p[1],p[0],:]*0.5 + nimg[p[1],p[0]-2,:]*0.5
                    #else:
                    #    print("no interp y,x:"+str(p[1]-1) +","+str(p[0]))
    return nimg.astype(np.uint8)


In [98]:
def cylin_proj(img, f):
    rows = img.shape[0]
    columns = img.shape[1]
    
    new_img = np.ones_like(img)
    #f = 400
    x_c = columns/2
    y_c = rows/2
    
    for x in range(columns):
        for y in range(rows):
            theta = math.atan((x- x_c )/ f)
            x_t = int(f * math.tan( (x-x_c) / f) + x_c)
            y_t = int( (y-y_c) / math.cos(theta) + y_c)
            
            if x_t >= columns or x_t < 0 or y_t >= rows or y_t < 0:
                continue
            else:
                new_img[y, x, :] = img[y_t, x_t, :]
    return new_img

In [99]:
#calculate Sigma values for different Gaussians
def sigmas():
    ma = np.zeros((octaves, scales))
    k = 2**(1./(scales-3))
    for i in range(0, octaves):
        for j in range(0, scales):
            ma[i][j] = (i+1) * sigma * (k**j)
    return ma

In [100]:
def gaussian_filter(sigma): 
    size = 2*np.ceil(3*sigma)+1 
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1] 
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2))) / (2*np.pi*sigma**2)
    return g/g.sum()

In [101]:
#calculate a gaussian kernel
#by default a radius will be chosen to, so kernel covers 99.7$ of data 
def gaussianKernel(SD, r):
    summat = 0
    if r == -1:
        r = int(3*SD)
    side = 2*r + 1
    return side

In [102]:
def extracteKeyP(DoG, o, s, size, Sigmas):
    if o == 0:
        amp = 0.5
    else:
        amp = 2**(o-1)
    P = []
    c_th = float(rCurvature+1) * (rCurvature + 1) / rCurvature
    edge = gaussianKernel(Sigmas[o][s], -1)
    c = 0
    for y in range(edge, size[1]-edge):
        for x in range(edge, size[0]-edge):
            sub0 = np.array([DoG[(scales-1)*o + s -1][x-1][y-1:y+2],DoG[(scales-1)*o + s -1][x][y-1:y+2],DoG[(scales-1)*o + s -1][x+1][y-1:y+2]])
            sub1 = np.array([DoG[(scales-1)*o + s][x-1][y-1:y+2],DoG[(scales-1)*o + s][x][y-1:y+2],DoG[(scales-1)*o + s][x+1][y-1:y+2]])
            sub2 = np.array([DoG[(scales-1)*o + s +1][x-1][y-1:y+2],DoG[(scales-1)*o + s +1][x][y-1:y+2],DoG[(scales-1)*o + s +1][x+1][y-1:y+2]])
            sub = np.append(sub0,sub1)
            sub = np.append(sub,sub2)
            sub = sub.flatten()
            if (sub1[1][1] > 0 and sub1[1][1] == max(sub)) or (sub1[1][1] < 0 and sub1[1][1] == min(sub)):
                if abs(sub1[1][1]) < contrastThreshold:
                    continue
                #calculate trace and determinant of hessian matrix
                Dx = sub1[2][1]/2. - sub1[0][1]/2.
                Dy = sub1[1][2]/2. - sub1[1][0]/2.
                Dz = sub2[1][1]/2. - sub0[1][1]/2.
                Dxx = sub1[0][1] + sub1[2][1] - 2*sub1[1][1]
                Dyy = sub1[1][0] + sub1[1][2] - 2*sub1[1][1]
                Dzz = sub2[1][1] + sub0[1][1] - 2*sub1[1][1]
                Dxy = sub1[0][2]/4. + sub1[2][0]/4. - sub1[0][0]/4. - sub1[2][2]/4.
                Dyz = sub0[1][2]/4. + sub2[1][0]/4. - sub0[1][0]/4. - sub2[1][2]/4.
                Dxz = sub0[2][1]/4. + sub2[0][1]/4. - sub0[0][1]/4. - sub2[2][1]/4.
                tr = (Dxx+Dyy) * (Dxx+Dyy)
                det = (Dxx*Dyy - Dxy*Dxy)
                df = np.matrix([[Dx], [Dy], [Dz]])
                ddf = np.matrix([[Dxx, Dxy, Dxz], [Dxy, Dyy, Dyz], [Dxz, Dyz, Dzz]])
                try:
                    x_m = np.linalg.inv(ddf).dot(df)*(-1)
                except np.linalg.LinAlgError:
                # Not invertible. Skip this one.
                    continue
                fx_m = sub1[1][1] + 0.5*(df.transpose().dot(x_m))
                
                if det >= 1e-3 and (tr/det) < c_th and abs(fx_m) > contrastThreshold:
                    if abs(x_m[0]) <= 0.5 and abs(x_m[1]) <= 0.5 and abs(x_m[2]) <= 0.5:
                        x_p = int(x * amp)
                        y_p = int(y * amp)
                        loc = (x_p, y_p, s)
                        P.append(loc)

    #print(c)
    return P

In [103]:
def cal_gra(G, o, s, x, y):
    m = (( ((G[int(scales)*o+s][x+1][y] - G[int(scales)*o+s][x-1][y]) ** 2) + ((G[int(scales)*o+s][x][y+1] - G[int(scales)*o+s][x][y-1]) ** 2) ) ** 0.5) 
    theta = (180 + (180 / np.pi) * (np.arctan2((G[int(scales-1)*o+s][x][y+1] - G[int(scales)*o+s][x][y-1]), (G[int(scales)*o+s][x+1][y] - G[int(scales)*o+s][x-1][y]))))        

    return (m, theta)

In [104]:
def SIFT(inputImage):
    Sigmas = sigmas()
    G = []
    for i in range(0, octaves):
        for j in range(0, scales):
            if i == 0:
                size = 2
                image = cv2.resize(inputImage, (inputImage.shape[1]*size,inputImage.shape[0]*size), interpolation=cv2.INTER_CUBIC)
            else:
                size = 2**(i-1)
                image = cv2.resize(G[scales*(i-1)+2].astype(np.uint8), (int(inputImage.shape[1]/size),int(inputImage.shape[0]/size)), interpolation=cv2.INTER_AREA)
            
            f = filters.gaussian_filter(image, Sigmas[i][j]).astype(np.int32)
            G.append(f)


    DoG = []#0 1 2 3 
    for i in range(0, octaves):
        for j in range(0, scales-1):
            DoG.append(G[scales*i + j + 1] - G[scales*i + j])


    print("extractkeypoint")
    P = []
    for i in range(0, octaves):#0 1 2 3
        print(i)
        for j in range(1, scales-2):#1 2 3 4
            sizeD = DoG[(scales-1)*i + j -1].shape
            p = extracteKeyP(DoG, i, j, sizeD, Sigmas)
            P.append(p)
    count = 0
    for p in P:
        count = count + len(p)
    print(count)
    #for p in P:
    #    for kp in p:
    #        cv2.line(img, (kp[1], kp[0]+4), (kp[1], kp[0]-4), (0, 255, 0), 2)
    #        cv2.line(img, (kp[1]+4, kp[0]), (kp[1]-4, kp[0]), (0, 255, 0), 2)
    #plt.figure()
    #plt.imshow(img)
    
    
    
    print("find max dir")
    kp_dir = []
    for o in range(0, octaves*(scales-3)):#0 1 2 3
        if int(o/(scales-3)) == 0:
            amp = 0.5
        else:
            amp = 2**(int(o/(scales-3))-1)
        if P[o] is not None:
            for kp in P[o]:
                ori_hist = np.zeros(36)
                sig = (1.5 * kp[2])
                gaussian_window = gaussian_filter(sig)
                w = int(2*np.ceil(sig)+1)
                for i in range(-w, w+1):
                    for j in range(-w, w+1):
                        if (i + kp[0]/amp < 0) or (i + kp[0]/amp >= G[(scales)*int(o/(scales-3))].shape[0] - 1) or (j + kp[1]/amp < 0) or (j + kp[1]/amp >= G[(scales)*int(o/(scales-3))].shape[1] - 1):
                            continue
                        m, theta = cal_gra(G, int(o/(scales-3)), int(kp[2]), int(i + kp[0]/amp), int(j + kp[1]/amp))
                        index = int(np.floor(theta/10))
                        if index == 36:
                            index = 0
                        ori_hist[index] = ori_hist[index] + m*gaussian_window[i, j]

                maxval = np.amax(ori_hist)
                maxidx = np.argmax(ori_hist)
                new_theta = maxidx*10+5
                kp_dir.append((kp[0], kp[1], new_theta, int(o/(scales-3)), kp[2]))
                
                ori_hist[maxidx] = 0
                maxidx = np.argmax(ori_hist)
                n_maxval = np.amax(ori_hist)
                if n_maxval > maxval*0.8:
                    new_theta = maxidx*10+5
                    kp_dir.append((kp[0], kp[1], new_theta, int(o/(scales-3)), kp[2]))
    #print(kp_dir)
    
    print("calculate discriptor")
    descriptors = []
    loc = []
    feature = []
    for kp in kp_dir:
        if kp[3] == 0:
            amp = 0.5
        else:
            amp = 2**(kp[3]-1)
        descriptor = []
        descriptor.append((kp[0], kp[1]))
        loc.append((kp[0], kp[1]))
        d = np.zeros(128)
        for x in range(-8, 8):
            for y in range(-8, 8):
                m, theta = cal_gra(G, int(kp[3]), int(kp[4]), int(kp[0]/amp + x), int(kp[1]/amp + y))
                mtheta = kp[2] * np.pi / 180.0
                x_r = Rround((np.cos(mtheta) * x) - (np.sin(mtheta) * y))
                y_r = Rround((np.sin(mtheta) * x) + (np.cos(mtheta) * y))
                gaussian_window = gaussian_filter(16./6)
                if (kp[0]/amp + x_r < 0) or (kp[0]/amp + x_r >= G[(scales)*kp[3]+kp[4]].shape[0] - 1) or (kp[1]/amp + y_r < 0) or (kp[1]/amp + y_r >= G[(scales)*kp[3]+kp[4]].shape[1] - 1):
                    continue
                x_pos_w = 1 - abs(x+0.5-((x+8)//4*4-6))/4.
                y_pos_w = 1 - abs(y+0.5-((y+8)//4*4-6))/4.
                
                m = m * gaussian_window[int(x_r), int(y_r)] 
                angle = theta - kp[2]
                if angle < 0:
                    angle = 360 + angle
                if angle >= 360:
                    angle = angle - 360
                
                index = int(np.floor((angle/45.)))
                xp = (x+8)//4
                yp = (y+8)//4
                d[index + xp*32 + yp*8] = d[index + xp*32 + yp*8] + m*(1-abs(angle-index*45)/45.)* x_pos_w * y_pos_w
                d[(index+1)%8 + xp*32 + yp*8] = d[(index+1)%8 + xp*32 + yp*8] + m*(1-abs(angle-(index+1)*45)/45.)* x_pos_w * y_pos_w
                
                if (x+0.5-(xp*4-6)) > 0 and xp+1<4:
                    nxp = xp+1
                    d[index + nxp*32 + yp*8] = d[index + nxp*32 + yp*8] + m*(1-abs(angle-index*45)/45.)*(1 - abs(x+0.5-(nxp*4-6))/4.)*(1 - abs(y+0.5-(yp*4-6))/4.)
                    d[(index+1)%8 + nxp*32 + yp*8] = d[(index+1)%8 + nxp*32 + yp*8] + m*(1-abs(angle-(index+1)*45)/45.)*(1 - abs(x+0.5-(nxp*4-6))/4.)*(1 - abs(y+0.5-(yp*4-6))/4.)
                
                elif (x+0.5-(xp*4-6)) < 0 and xp-1>=0:
                    nxp = xp-1
                    d[index + nxp*32 + yp*8] = d[index + nxp*32 + yp*8] + m*(1-abs(angle-index*45)/45.)*(1 - abs(x+0.5-(nxp*4-6))/4.)*(1 - abs(y+0.5-(yp*4-6))/4.)
                    d[(index+1)%8 + nxp*32 + yp*8] = d[(index+1)%8 + nxp*32 + yp*8] + m*(1-abs(angle-(index+1)*45)/45.)*(1 - abs(x+0.5-(nxp*4-6))/4.)*(1 - abs(y+0.5-(yp*4-6))/4.)
                
                if (y+0.5-(yp*4-6)) >0 and yp+1<4:
                    nyp = yp+1
                    d[index + xp*32 + nyp*8] = d[index + xp*32 + nyp*8] + m*(1-abs(angle-index*45)/45.)*(1 - abs(x+0.5-(xp*4-6))/4.)*(1 - abs(y+0.5-(nyp*4-6))/4.)
                    d[(index+1)%8 + xp*32 + nyp*8] = d[(index+1)%8 + xp*32 + nyp*8] + m*(1-abs(angle-(index+1)*45)/45.)*(1 - abs(x+0.5-(xp*4-6))/4.)*(1 - abs(y+0.5-(nyp*4-6))/4.)
                    
                elif (y+0.5-(yp*4-6)) <0 and yp-1>=0:
                    nyp = yp-1
                    d[index + xp*32 + nyp*8] = d[index + xp*32 + nyp*8] + m*(1-abs(angle-index*45)/45.)*(1 - abs(x+0.5-(xp*4-6))/4.)*(1 - abs(y+0.5-(nyp*4-6))/4.)
                    d[(index+1)%8 + xp*32 + nyp*8] = d[(index+1)%8 + xp*32 + nyp*8] + m*(1-abs(angle-(index+1)*45)/45.)*(1 - abs(x+0.5-(xp*4-6))/4.)*(1 - abs(y+0.5-(nyp*4-6))/4.)
                
                
        d = d/np.linalg.norm(d)
        np.clip(d, 0, 0.2)
        d_nor = d/np.linalg.norm(d)
        
        descriptor.append(d_nor)
        feature.append(d_nor)
        descriptors.append(descriptor)
    #print(loc)
    #print(feature)
    return (loc, feature)

In [105]:
def panorama(inputname):
    read_directory(inputname)
    #use: len(array_of_img) for looping the image, array_of_img[0], array_of_img[1],array_of_img[2],...for processing each image
    #Start SIFT here
    imageoutput = array_of_img[0]
    size = imageoutput.shape
    #imageout = []
    re = 1
    #for i in range(0, len(array_of_img)):
        #array_of_img[i] = cylin_proj(array_of_img[i], 95)
    for i in range(1, len(array_of_img)):#len(array_of_img)
        print("**********image"+str(i-1)+"and image"+str(i)+"**************")
        #use SIFT compute all keypoints from each images
        s=imageoutput.shape
        imageA = array_of_img[i]#A is right image
        imageB = imageoutput[:, s[1]-size[1]: ,:]#B is left image
        print(imageB.shape)
        #detector = cv2.xfeatures2d.SIFT_create()
        #(keypointsA, featuresA) = detector.detectAndCompute(imageA, None)
        #detector2 = cv2.xfeatures2d.SIFT_create()
        #(keypointsB, featuresB) = detector2.detectAndCompute(imageB, None)
        if re == 1:
            grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        else:
            reA = cv2.resize(imageA, (int(size[1]/2),int(size[0]/2)), interpolation=cv2.INTER_AREA)
            grayA = cv2.cvtColor(reA, cv2.COLOR_BGR2GRAY)
        (keypointsA, featuresA) = SIFT(grayA)
        if i == 1 and len(keypointsA) > 3000:
            re = 2
            reA = cv2.resize(imageA, (int(size[1]/2),int(size[0]/2)), interpolation=cv2.INTER_AREA)
            grayA = cv2.cvtColor(reA, cv2.COLOR_BGR2GRAY)
            (keypointsA, featuresA) = SIFT(grayA)
        print(grayA.shape)
        if re ==1:
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        else:
            reB = cv2.resize(imageB, (int(size[1]/2),int(size[0]/2)), interpolation=cv2.INTER_AREA)
            grayB = cv2.cvtColor(reB, cv2.COLOR_BGR2GRAY)
        print(grayB.shape)
        (keypointsB, featuresB) = SIFT(grayB)
        #kA = np.float32([kp.pt for kp in keypointsA])
        #kB = np.float32([kp.pt for kp in keypointsB])
        Homo = findHomo(keypointsA, keypointsB, featuresA, featuresB,grayA.shape[1], re)
        result = warpTransport2(imageA,imageB, Homo.T, imageB.shape[0])
        for n in range(imageB.shape[1], result.shape[1]):
            if max(result[0:int(result.shape[0]/3),n,0]) == 0 and max(result[0:int(result.shape[0]/3),n,1]) == 0 and max(result[0:int(result.shape[0]/3),n,2]) == 0:
                break
            elif max(result[int(2*result.shape[0]/3):,n,0]) == 0 and max(result[int(2*result.shape[0]/3):,n,1]) == 0 and max(result[int(2*result.shape[0]/3):,n,2]) == 0:
                break
        imageoutput = np.concatenate(( imageoutput[:, :s[1]-size[1] ,:],result[0:imageoutput.shape[0], 0:n,:]), axis=1)
        
    
    #End of SIFT here and use imageoutput for your output
    #imageoutput = keypoints
    array_of_img.clear()
    return imageoutput

In [123]:
f = open('testfile.txt', 'r')
dirname = str(f.readline()).strip()
while(dirname):
    imageoutput = panorama(dirname)
    b, g, r = cv2.split(imageoutput)
    imageout = cv2.merge([r, g, b])
    #plt.figure()
    #plt.imshow(imageout)
    cv2.imwrite(dirname+".jpg", imageoutput)
    dirname = str(f.readline()).strip()

**********image0and image1**************
(460, 1023, 3)
extractkeypoint
0
1
2
1876
find max dir
calculate discriptor
(460, 345)
(460, 1023)
extractkeypoint
0
1
2
2999
find max dir
calculate discriptor
(2378, 3812)
198
[[  1.   0.  25.]
 [  0.   1. 931.]
 [  0.   0.   1.]]
[4.850e+02 1.276e+03 1.000e+00]
[4.860e+02 1.368e+03 1.000e+00]
0
**********image1and image2**************
(460, 1023, 3)
extractkeypoint
0
1
2
884
find max dir
calculate discriptor
(460, 254)
(460, 1023)
extractkeypoint
0
1
2
4260
find max dir
calculate discriptor
(1139, 5360)
103
[[  1.   0.  39.]
 [  0.   1. 938.]
 [  0.   0.   1.]]
[4.990e+02 1.192e+03 1.000e+00]
[5.000e+02 1.277e+03 1.000e+00]
0
**********image2and image3**************
(460, 1023, 3)
extractkeypoint
0
1
2
410
find max dir
calculate discriptor
(460, 311)
(460, 1023)
extractkeypoint
0
1
2
4654
find max dir
calculate discriptor
(554, 5850)
92
[[ 1.00000000e+00 -2.94117647e-02  3.22352941e+01]
 [-2.94117647e-02  1.00000000e+00  9.56794118e+02]
 [ 0.0

In [107]:
array_of_img.clear()